# Test out Hilbert Curve 2d

> 

In [ ]:
import os, json, re

import numpy as np
import pandas as pd

# import plotly.express as px
# import plotly.io as pio
# pio.templates.default = "plotly_white"
# from sklearn.manifold import TSNE # for visualizing embeddings

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna  import *
from EnvDL.dlfn import * # includes LSUV_

from tqdm import tqdm

In [ ]:
use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F # F.mse_loss

import einops # for einops.rearrange

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/03.12_g2fc_W_Hilbert_conv2d_resnet.ipynb/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

### Weather (Variable in season)

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'

np.load(load_from+'PlantHarvestNames.npy')
PlantHarvest = np.load(load_from+'PlantHarvest.npy')

# WMat = np.load(load_from+'WMat.npy')
WMatNames = np.load(load_from+'WMatNames.npy')

WMat_hilb = np.load(load_from+'WMat_hilb.npy')

### Response and lookup

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx  Geno_Idx  Is_Phno_Idx
obs_env_lookup = np.load(load_from+'obs_env_lookup.npy')   # Phno_Idx  Env_Idx   Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')

## Demo retrieval

In [ ]:
# # Indexes
# idx = 0

# geno_idx = obs_geno_lookup[idx, 1]
# env_idx = obs_env_lookup[idx, 1]

In [ ]:
# # Response
# YMat[idx]

In [ ]:
# # Weather (plant in ground) ------------------------------------------------------------------------
# # generate planting information
# ## Basic tensor
# WPlant = np.zeros(365)
# WPlant[PlantHarvest[idx, 0]:PlantHarvest[idx, 1]] = 1

# ## Hilbert
# WPlant_hilb = np_3d_to_hilbert(WPlant[None, :, None])
# WPlant_hilb = WPlant_hilb.squeeze(axis = 3)
# WPlant_hilb[np.isnan(WPlant_hilb)] = 0
# # px.imshow(WPlant_hilb.squeeze())
# # compare with
# # px.imshow(WMat_hilb[env_idx][8, :, :])

In [ ]:
# # Weather (all) ------------------------------------------------------------------------------------
# ## Basic tensor
# np.concatenate([WMat[env_idx], WPlant[None, :]], axis = 0).shape
# ## Hilbert
# np.concatenate([WMat_hilb[env_idx], WPlant_hilb], axis = 0).shape

### Set up Dataset

In [ ]:
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)

temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)

In [ ]:
train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
# Debugging Data -----------------------------------------------------------------------------------

train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
training_dataloader = DataLoader(BigDataset(
    y = torch.from_numpy(                                 y_cs[train_idx])[:, None].to(torch.float).to('cuda'),
    lookup_obs = torch.from_numpy(np.asarray([i for i in range(train_idx.size)])),
    lookup_env = torch.from_numpy(   np.asarray(obs_env_lookup[train_idx])),
    W = torch.from_numpy(WMat_hilb).to(torch.float).to('cuda'),
    P = torch.from_numpy(PlantHarvest).to(torch.int),
    W_type = 'hilbert'
    ),
    batch_size = 50,
    shuffle = True
)

validation_dataloader = DataLoader(BigDataset(
    y = torch.from_numpy(                                 y_cs[test_idx])[:, None].to(torch.float).to('cuda'),
    lookup_obs = torch.from_numpy(np.asarray([i for i in range(test_idx.size)])),
    lookup_env = torch.from_numpy(np.asarray(   obs_env_lookup[test_idx])),
    W = torch.from_numpy(WMat_hilb).to(torch.float).to('cuda'),
    P = torch.from_numpy(PlantHarvest),
    W_type = 'hilbert'
    ),
    batch_size = 50,
    shuffle = False 
)


In [ ]:

# ResNet2d(
#         block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
#         layers = [2, 2, 2, 2], #: List[int],
#         # num_classes: int = 1000,
#         zero_init_residual = False,
#         groups = 1,
#         width_per_group = 64,
#         replace_stride_with_dilation = None,
#         norm_layer = None,
#         input_channels = 17
#     ).to('cuda')(next(iter(training_dataloader))[1])



In [ ]:


# Module for training subnetworks.
class plDNN_wthr(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        y_i, w_i = batch
        pred = self.mod(w_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        y_i, w_i = batch
        pred = self.mod(w_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    


In [ ]:
model = ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [2, 2, 2, 2], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None,
        input_channels = 17
    ).to('cuda')


max_epoch = 10
DNNW = plDNN_wthr(model)     
optimizer = DNNW.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="w-hilb-res-4rep2-from-pytorch")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNW, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
model = ResNet2d(
        block = BasicBlock2d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [4, 4, 4, 4], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None,
        input_channels = 17
    ).to('cuda')


max_epoch = 10
DNNW = plDNN_wthr(model)     
optimizer = DNNW.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="w-hilb-res-4rep4-from-pytorch")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNW, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: tb_logs/w-hilb-res-4rep4-from-pytorch
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type     | Params

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
